# Phenotype Data

## Library Import

In [ ]:
# import json
# import os

# import matplotlib.pyplot as plt
# import numpy as np
# import pandas as pd
# import scanpy as sc
# import seaborn as sns
import torch

# import torch_geometric
from matplotlib import colors as mcolors

In [ ]:
# import argparse
import json
import os

import numpy as np
import pandas as pd
import scanpy as sc
from sklearn.model_selection import train_test_split
from tqdm import tqdm

CANCER_TYPES = [
    "Adenocarcinoma",
    "Squamous cell carcinoma",
    "Adeno squamous cell carcinoma",
    "Control",
    "Large cell carcinoma",
    "Large cell neuroendocrine carcinoma",
    "NSCLC",
    "Mesotheliom",
    "Basaloides Ca",
    "NA",
]

TARGETS = [
    "Relapse",
    "Ev.O",
    "Grade",
    "Stage",
    "T.new",
    "N",
    "M.new",
    "R",
    "Chemo",
    "Radio",
    "DFS",
    "OS",
    "LN.Met",
    "Dist.Met",
]

IMC_TO_CODEX_SCALE_FACTOR = 2.75

## Helper Functions

## NSCLC Dataset

In [ ]:
adata_raw = sc.read_h5ad("../data/phenotype/sce_all_annotated.h5ad")
adata_raw

In [ ]:
def filter_for_cancer_types(cell_information, cancer_types):
    assert set(cancer_types).issubset(set(CANCER_TYPES[:3]))
    return cell_information.loc[cell_information["DX.name"].isin(cancer_types)]


def load_adata(adata_path, cancer_types, targets):
    # load adata and cell information
    adata = sc.read_h5ad(adata_path)
    cell_information = adata.obs

    # replace -2147483648 with NaN
    cell_information_clean = cell_information.replace(-2147483648, np.nan)

    # filter for cancer types
    cell_information_clean = cell_information_clean.loc[
        (cell_information_clean["DX.name"] == "Adenocarcinoma")
        | (cell_information_clean["DX.name"] == "Squamous cell carcinoma")
        | (cell_information_clean["DX.name"] == "Adeno squamous cell carcinoma")
    ]
    cell_information_clean = filter_for_cancer_types(cell_information_clean, cancer_types)

    # normalize area
    cell_information_clean["Area"] = cell_information_clean["Area"] / cell_information_clean[
        "Area"
    ].quantile(0.95)

    # lower cell category, type and subtype (e.g. "T cell" -> "t cell")
    cell_information_clean["cell_category"] = cell_information_clean["cell_category"].str.lower()
    cell_information_clean["cell_type"] = cell_information_clean["cell_type"].str.lower()
    cell_information_clean["cell_subtype"] = cell_information_clean["cell_subtype"].str.lower()

    # drop NaNs in cell category, type and subtype
    cell_information_clean = cell_information_clean.dropna(
        subset=["cell_category", "cell_type", "cell_subtype"]
    )

    # convert LN.Met, Dist.Met, and NeoAdj to binary
    cell_information_clean["LN.Met"] = cell_information_clean["LN.Met"].cat.rename_categories(
        {"No LN Metastases": 0, "LN Metastases": 1}
    )
    cell_information_clean["Dist.Met"] = cell_information_clean["Dist.Met"].cat.rename_categories(
        {"No Dist. Metastases": 0, "Dist. Metastases": 1}
    )
    cell_information_clean["NeoAdj"] = cell_information_clean["NeoAdj"].cat.rename_categories(
        {"NoNeoAdjuvantTherapy": 0, "NeoAdjuvantTherapy": 1}
    )

    # drop NaNs in target columns
    cell_information_clean = cell_information_clean.dropna(subset=targets)

    return adata, cell_information_clean

In [ ]:
adata_path = "../data/phenotype/sce_all_annotated.h5ad"
target_names = ["Relapse", "Ev.O", "LN.Met"]

adata, cell_information_clean = load_adata(adata_path, CANCER_TYPES[:3], target_names)

In [ ]:
cell_information_clean

In [ ]:
cell_information_clean["Tma_ac"]

In [ ]:
def separate_regions(adata, cell_information, targets):
    # group regions by Tma_ac
    grouped = cell_information.groupby("Tma_ac", observed=True)

    # get the biomarker names = proteins measured
    biomarker_names = adata.var_names.values

    # initialize the regions and targets dictionaries
    # regions = {region_name: {"cell_ids": cell_ids, "coordinates": coordinates, "cell_types": cell_types, "sizes": sizes, "biomarkers": biomarkers}}
    # targets = {target1: [target_value1, target_value2, ...], target2: [target_value1, target_value2, ...], ...}
    regions = {}
    targets = {target: [] for target in targets}

    # iterate over the groups and extract the information for each region
    for name, group in grouped:
        cell_ids = group.index.values
        coordinates = group[["Center_X", "Center_Y"]].values
        cell_types = group["cell_subtype"].values
        sizes = group["Area"].values
        biomarkers = adata[cell_ids].layers["c_counts_asinh_scaled"].toarray()
        for target in targets:
            target_values = group[target]
            unique_targets = target_values.unique()
            assert len(unique_targets) == 1
            targets[target].append(unique_targets[0])
        regions[name] = {
            "cell_ids": cell_ids,
            "coordinates": coordinates,
            "cell_types": cell_types,
            "sizes": sizes,
            "biomarkers": biomarkers,
        }

    return regions, targets, biomarker_names

In [ ]:
regions, targets, biomarker_names = separate_regions(adata, cell_information_clean, target_names)

In [ ]:
def train_valid_split(cell_information, target_columns, valid_fraction):
    # concat all target columns to one string used for stratification
    cell_information["stratify"] = (
        cell_information[target_columns].astype(str).agg("".join, axis=1)
    )

    # split patients into train and valid sets using stratified sampling
    patient_df = cell_information[["Patient_ID", "stratify"]].drop_duplicates()
    train_patients, valid_patients = train_test_split(
        list(patient_df["Patient_ID"]),
        test_size=valid_fraction,
        stratify=list(patient_df["stratify"]),
        random_state=42,
    )

    # get the regions for train and valid sets
    train_regions = cell_information.loc[cell_information["Patient_ID"].isin(train_patients)]
    train_regions = list(train_regions["Tma_ac"].unique())
    valid_regions = cell_information.loc[cell_information["Patient_ID"].isin(valid_patients)]
    valid_regions = list(valid_regions["Tma_ac"].unique())

    return train_regions, valid_regions

In [ ]:
valid_fraction = 0.2
output_dir = "../data/phenotype/regions"

train_regions, valid_regions = train_valid_split(
    cell_information_clean, ["Relapse", "Grade"], valid_fraction
)

os.makedirs(output_dir, exist_ok=True)
with open(os.path.join(output_dir, "train_regions_grade.json"), "w") as f:
    json.dump(train_regions, f)
with open(os.path.join(output_dir, "valid_regions_grade.json"), "w") as f:
    json.dump(valid_regions, f)

In [ ]:
def store_regions(regions, targets, biomarker_names, output_dir):
    for name, region in tqdm(list(regions.items())):
        os.makedirs(os.path.join(output_dir, name), exist_ok=True)
        cell_ids = region["cell_ids"]
        # store cell coordinates as csv with cell_ids
        cell_coords = region["coordinates"]
        cell_coords *= IMC_TO_CODEX_SCALE_FACTOR
        coordinates_df = pd.DataFrame(cell_coords, columns=["X", "Y"])
        coordinates_df["CELL_ID"] = cell_ids
        coordinates_df.to_csv(os.path.join(output_dir, name, "coordinates.csv"), index=False)
        # store cell types as csv with cell_ids
        cell_types = region["cell_types"]
        cell_types_df = pd.DataFrame(cell_types, columns=["CELL_TYPE"])
        cell_types_df["CELL_ID"] = cell_ids
        cell_types_df.to_csv(os.path.join(output_dir, name, "cell_types.csv"), index=False)
        # store cell sizes as csv with cell_ids
        cell_sizes = region["sizes"]
        cell_sizes_df = pd.DataFrame(cell_sizes, columns=["SIZE"])
        cell_sizes_df["CELL_ID"] = cell_ids
        cell_sizes_df.to_csv(os.path.join(output_dir, name, "cell_sizes.csv"), index=False)
        # store biomarker expression as csv with cell_ids
        biomarkers = region["biomarkers"]
        biomarkers_df = pd.DataFrame(biomarkers, columns=biomarker_names)
        biomarkers_df["CELL_ID"] = cell_ids
        biomarkers_df.to_csv(os.path.join(output_dir, name, "expression.csv"), index=False)
    # store target values as csv, columns are target keys, rows are regions
    targets_df = pd.DataFrame(targets)
    targets_df.columns = [col.upper() for col in targets_df.columns]
    targets_df["REGION_ID"] = list(regions.keys())
    targets_df.to_csv(os.path.join(output_dir, "targets.csv"), index=False)

In [ ]:
store_regions(regions, targets, biomarker_names, output_dir)

## Graphs

In [ ]:
example_graph = torch.load("../data/phenotype/raw/175A_119.0.gpt", weights_only=False)
example_graph

In [ ]:
example_graph.edge_attr[:, 1].unique()

In [ ]:
cell_types = example_graph.x[:, 0].long()

In [ ]:
targets_path = os.path.join("../data/phenotype/raw", "targets.csv")
if not os.path.exists(targets_path):
    raise FileNotFoundError(f"File {targets_path} does not exist. Please provide the targets.")
targets = pd.read_csv(targets_path)

In [ ]:
targets.loc[targets["REGION_ID"] == "175A_119"]

In [ ]:
graph_tasks = ["RELAPSE", "EV.O", "LN.MET"]

# append the targets to the graph
example_graph.y = torch.zeros((len(graph_tasks)), dtype=torch.float32)
for i, task in enumerate(graph_tasks):
    if task not in targets.columns:
        raise ValueError(f"Task {task} not found in targets.")
    example_graph.y[i] = torch.tensor(
        targets.loc[targets["REGION_ID"] == example_graph.region_id, task].values
    )
    example_graph.y[i] = example_graph.y[i].to(torch.float32)

In [ ]:
example_graph.y

In [ ]:
json_path = "../data/phenotype/json_files"

ct_mapping_path = os.path.join(json_path, "cell_type_mapping.json")
ct_freq_path = os.path.join(json_path, "cell_type_freq.json")
biomarkers_path = os.path.join(json_path, "biomarkers.json")
with open(ct_mapping_path, "r") as f:
    cell_type_mapping = json.load(f)
with open(ct_freq_path, "r") as f:
    cell_type_freq = json.load(f)
with open(biomarkers_path, "r") as f:
    biormarkers = json.load(f)

In [ ]:
cell_type_freq

In [ ]:
cell_type_mapping

In [ ]:
biormarkers

## Testing Code

In [ ]:
import rootutils

rootutils.setup_root(os.getcwd(), indicator=".project-root", pythonpath=True)

from src.data.cellular_graph_datamodule import (
    CellularGraphDataModule,
    CellularGraphDataset,
)

In [ ]:
graph_dir = "../data/phenotype/raw"
regions = []
for file in os.listdir(graph_dir):
    if file.endswith(".gpt"):
        regions.append(file.split(".")[0])
json_path = "../data/phenotype/json_files"
num_subgraphs = 5
subgraph_size = 3
sampling_avoid_unassigned = (True,)
unassigned_cell_type = "Unassigned"

data_dir = "../data/phenotype/raw"
processed_dir = "../data/phenotype/processed"
batch_size = 2
num_workers = 4
pin_memory = False
json_path = "../data/phenotype/json_files"
num_subgraphs = 5
subgraph_size = 3
sampling_avoid_unassigned = True
unassigned_cell_type = "Unassigned"
graph_tasks = ["RELAPSE", "EV.O", "LN.MET"]
redo_preprocess = True
seed = 42

In [ ]:
data_set = CellularGraphDataset(
    regions=regions,
    graph_dir=processed_dir,
    json_path=json_path,
    num_subgraphs=num_subgraphs,
    subgraph_size=subgraph_size,
    sampling_avoid_unassigned=sampling_avoid_unassigned,
    unassigned_cell_type=unassigned_cell_type,
    seed=42,
)

In [ ]:
data_set

In [ ]:
data_set[0]

In [ ]:
data_module = CellularGraphDataModule(
    data_dir=data_dir,
    processed_dir=processed_dir,
    batch_size=batch_size,
    num_workers=num_workers,
    pin_memory=pin_memory,
    json_path=json_path,
    num_subgraphs=num_subgraphs,
    subgraph_size=subgraph_size,
    sampling_avoid_unassigned=sampling_avoid_unassigned,
    unassigned_cell_type=unassigned_cell_type,
    graph_tasks=graph_tasks,
    redo_preprocess=redo_preprocess,
    seed=seed,
)

In [ ]:
data_module

In [ ]:
data_module.prepare_data()

In [ ]:
data_module.setup()

In [ ]:
data_module.dataset

In [ ]:
dataset = data_module.dataset
dataset[0]

In [ ]:
train_loader = data_module.train_dataloader()
b = None
for batch in train_loader:
    b = batch
    print(batch)
    break

In [ ]:
b

In [ ]:
len(train_loader)